# Data Preparations

In [ ]:
import pandas as pd
import re
import numpy as np 
from sklearn.metrics.pairwise import cosine_similarity
import matplotlib
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import re
import numpy as np 
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import MinMaxScaler

%matplotlib inline

In [ ]:
data1 = pd.read_csv("Songs_with_features.csv")
data2 = pd.read_csv("spotify.csv")
print(data1.shape)
print(data2.shape)

In [ ]:
print(data1.columns)
print(data2.columns)


In [ ]:
data1.drop(columns=['Unnamed: 0', 'Unnamed: 0.1', 'length','danceability.1','tempo', 'time_signature'],inplace=True)
data2.drop(columns=['duration_ms','explicit', 'id','key','mode','tempo','valence'],inplace=True)

In [ ]:
data2.info()
data1.info()

In [ ]:
data1['release_date'] = pd.DatetimeIndex(data1['release_date']).year
data2['release_date'] =  pd.DatetimeIndex(pd.to_datetime(data2.year,format="%Y")).year
data2["artist"] = data2["artists"].apply(lambda x: x.replace('[','').replace(']','').replace("'",'').replace('"','')).astype(str)

data1 = data1[["name","artist","release_date","popularity","danceability","acousticness","energy","instrumentalness","liveness","loudness","speechiness"]]
data2 = data2[["name","artist","release_date","popularity","danceability","acousticness","energy","instrumentalness","liveness","loudness","speechiness"]]

In [ ]:
data1.head()

In [ ]:
data2.head()

In [ ]:
data1 = data1[data1['release_date'] > 1995]
data2 = data2[data2['release_date'] > 1995]
print(data1.shape)
print(data2.shape)

In [ ]:
data = pd.concat([data1,data2])
print(data.shape)
data.drop_duplicates(subset="name",inplace=True)

In [ ]:
print(data.shape)
data.describe()

In [ ]:
data = data.sort_values(by=["release_date"],ascending=False,ignore_index=True)

In [ ]:
data.head()

In [ ]:
data.to_csv("Spotify-Songs-Data.csv")

# Exploraotry Data Analysis

In [ ]:
data = pd.read_csv("Spotify-Songs-Data.csv")

correlation = data.corr()
sns.heatmap(correlation,cmap='BrBG_r',annot_kws={"fontsize":12}, annot = True)


In [ ]:
popular_tracks = data.groupby('name')[['popularity']].mean().sort_values('popularity', ascending = False).reset_index().head(20)
popular_tracks

In [ ]:

sns.barplot(x = 'popularity', y = 'name', data = popular_tracks).set(title = 'Top Tracks vs Average Popularity');

In [ ]:
popular_artists = data.groupby('artist')[['popularity']].sum().sort_values('popularity', ascending = False).reset_index().head(20)
popular_artists 

In [ ]:
acousticness = data.groupby('release_date')[['acousticness']].mean().reset_index().sort_values('release_date')
energy = data.groupby('release_date')[['energy']].mean().reset_index().sort_values('release_date')
danceability = data.groupby('release_date')[['danceability']].mean().reset_index().sort_values('release_date')

sns.lineplot(x = 'release_date', y = 'acousticness', data = acousticness, color = 'purple').set(title = 'Average values of energy for every year')
sns.lineplot(x = 'release_date', y = 'energy', data = energy, color = 'red').set(title = 'Average values of energy for every year')
sns.lineplot(x = 'release_date', y = 'danceability', data = danceability, color = 'yellow').set(title = 'Average values of energy for every year')

plt.legend(['acousticness','energy','danceability']);

In [ ]:
# total_time = data_new_df.duration_minutes.sum() / (24*365*60)

# Feature Exrtac & Cosine Similarity Matrix

In [ ]:
data = pd.read_csv("Spotify-Songs-Data.csv")

In [ ]:
data.head()

In [ ]:
columns = ["release_date","popularity","danceability","acousticness","energy","instrumentalness","liveness","loudness","speechiness"]

scaler = MinMaxScaler()

for i in columns:
  data[i] = scaler.fit_transform(np.array(data[i]).reshape(-1,1))

In [ ]:
data.shape
data.head()

In [ ]:
datascaled = data[columns][:int((data.shape[0]-1)/2)]
datascaledarr = np.array(datascaled)
print(datascaledarr)
print(len(datascaledarr))

In [ ]:
# vsm  = cosine_similarity(datascaledarr,datascaledarr)
# np.save('vsm.npy', vsm) 

In [ ]:
vsm1  = cosine_similarity(datascaledarr,datascaledarr) 

In [ ]:
vsm1array = np.array(vsm1)
print((vsm1array.size * vsm1array.itemsize)/1000000000)
print(vsm1array.nbytes/1000000000)

# Recomendation

In [ ]:
name = "Blinding Lights"
songdata = data[data.name==name]
songdataidx = data[data.name==name].index[0]
songdatascaled = datascaledarr[songdataidx]
# features = [songdata[i].values[0] for i in columns]
# print(songdatascaled)
# print(songdataidx)

cs_data = list(enumerate(vsm2[songdataidx]))
print(cs_data)
cs_data.sort(key=lambda x:x[1],reverse=True)

In [ ]:
# print(cs_data[:10])
num = 1
for i in cs_data[0:25]:
  name2 = data["name"][i[0]] 
  artist = data["artist"][i[0]]
  print(num," ",name2,"by",artist)
  num +=1
